[View in Colaboratory](https://colab.research.google.com/github/Gauravv97/Tensorflow-deeplabV3-TrainSaveResume-on-Android/blob/master/Train_and_Resume_.ipynb)

In [0]:
import os
from os.path import expanduser
import tensorflow as tf 
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

home = expanduser("~")
os.chdir(home)
if not (os.path.exists('''models''')):
  !git clone https://github.com/tensorflow/models.git

In [0]:
!ls

In [0]:
os.chdir(home+'''/models/research/''')
!python slim/setup.py build
!python slim/setup.py install

In [0]:
os.chdir(home+"/models/research")
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
!cp -r slim/* deeplab

In [0]:
os.chdir(home+"/models/research/deeplab/datasets")
!sh download_and_convert_voc2012.sh

In [0]:
!mkdir -p /content/models/research/deeplab/datasets/pascal_voc_seg/init_models
!mkdir -p /content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set/train
!mkdir -p /content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set/eval
!mkdir -p /content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set/vis
!mkdir -p /content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set/export
os.chdir('/content/models/research/deeplab/datasets/pascal_voc_seg/init_models')
!wget -nd -c "http://download.tensorflow.org/models/deeplabv3_pascal_train_aug_2018_01_04.tar.gz"
!tar -xf "deeplabv3_pascal_train_aug_2018_01_04.tar.gz"
os.chdir('/content/models/research/')

In [0]:
!ls

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


folder_metadata = {
    'title' : 'train',
    # The mimetype defines this new file as a folder, so don't change this.
    'mimeType' : 'application/vnd.google-apps.folder'
}
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
flag=0
for file1 in file_list:
    if file1['title'] == 'train':
        folder_id = file1['id']
        flag=1
if flag==0:
  folder = drive.CreateFile(folder_metadata)
  folder.Upload()
  folder_title = folder['title']
  folder_id = folder['id']



files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
  file = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": folder_id}]})
  print("uploading "+f)
  file.SetContentFile(f)
  file.Upload()

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

folder_metadata = {
    'title' : 'train',
    # The mimetype defines this new file as a folder, so don't change this.
    'mimeType' : 'application/vnd.google-apps.folder'
}
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
flag=0
for file1 in file_list:
    if file1['title'] == 'train':
        folder_id = file1['id']
        flag=1
if flag==0:
  folder = drive.CreateFile(folder_metadata)
  folder.Upload()
  folder_title = folder['title']
  folder_id = folder['id']


In [0]:
'''Remove all previous checkpoint and train files if exists '''
!rm -r /content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set/train/*

In [0]:
!ls /content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set/train/

In [0]:
os.chdir('/content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set/train')
file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder_id)}).GetList()
for file1 in sorted(file_list, key = lambda x: x['title']):
    file1.GetContentFile(file1['title'])


In [0]:
!python /content/models/research/deeplab/train.py \
  --logtostderr \
  --train_split="trainval" \
  --model_variant="xception_65" \
  --atrous_rates=6 \
  --atrous_rates=12 \
  --atrous_rates=18 \
  --output_stride=16 \
  --decoder_output_stride=4 \
  --train_crop_size=1025 \
  --train_crop_size=1025 \
  --train_batch_size=1 \
  --training_number_of_steps=20 \
  --fine_tune_batch_norm=false \
  --tf_initial_checkpoint="/content/models/research/deeplab/datasets/pascal_voc_seg/init_models/deeplabv3_pascal_train_aug/model.ckpt" \
  --train_logdir="/content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set/train" \
  --dataset_dir="/content/models/research/deeplab/datasets/pascal_voc_seg/tfrecord"



''' Delete All old train files in Google Drive and upload new ones'''
os.chdir('/content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set/train')
file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder_id)}).GetList()
for file1 in sorted(file_list, key = lambda x: x['title']):
    file1.Delete() 
files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
  file = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": folder_id}]})
  print("uploading "+f)
  file.SetContentFile(f)
  file.Upload()

In [0]:
!ls /content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set/train/

In [0]:
!ls

In [0]:
os.chdir('/content/models/research/deeplab')
!python export_model.py \
  --logtostderr \
  --checkpoint_path="/content/models/research/deeplab/datasets/pascal_voc_seg/exp/train_on_trainval_set/train/model.ckpt-200" \
  --export_path="/content/models/research/deeplab/frozen_inference_graph.pb" \
  --model_variant="xception_65" \
  --atrous_rates=6 \
  --atrous_rates=12 \
  --atrous_rates=18 \
  --output_stride=16 \
  --decoder_output_stride=4 \
  --num_classes=21 \
  --crop_size=1025 \
  --crop_size=1025 \
  --inference_scales=1.0

In [0]:
import os
from io import BytesIO
import tarfile
import tempfile
import tensorflow as tf
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

from google.colab import files

In [0]:
SAMPLE_IMAGE = files.upload()

In [0]:
INPUT_TENSOR_NAME = 'ImageTensor:0'
OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
INPUT_SIZE = 1025
FROZEN_GRAPH_NAME = 'frozen_inference_graph'

graph = tf.Graph()

graph_def = None

file=open("/content/models/research/deeplab/frozen_inference_graph.pb",'r')
graph_def = tf.GraphDef.FromString(file.read())
if graph_def is None:
      raise RuntimeError('Cannot find inference graph.')
with graph.as_default():
      tf.import_graph_def(graph_def, name='')
sess = tf.Session(graph=graph)


def run(image):
    """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    width, height = image.size
    resize_ratio = 1.0 * INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = sess.run(
        OUTPUT_TENSOR_NAME,
        feed_dict={INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    return resized_image, seg_map

  



def getImage(img):
  """Inferences DeepLab model and visualizes result."""
  
  f = open(str(SAMPLE_IMAGE.keys()[0]),'r')
  jpeg_str = f.read()
  original_im = Image.open(BytesIO(jpeg_str))
  resized_im, seg_map = run(original_im)
  return resized_im, seg_map

resized_im, seg_map=getImage(SAMPLE_IMAGE)

In [0]:
import cv2
blurImg = cv2.blur(np.asarray(resized_im),(10,10)) 
plt.imshow(blurImg)
rMap=resized_im.load()
portrait=blurImg
pMap=blurImg
cni=0
cnj=0
for i in range(resized_im.size[1]-1):
    for j in range(resized_im.size[0]-1):
      if not seg_map[i][j]==0:
        cni=i
        cnj=j
        pMap[i,j]=rMap[j,i]


cv2.imshow('image',pMap)
